# Multiagent Pattern - Multiagent Collaboration

<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together. 

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.


In [2]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%%python3` not found.


In [3]:
import sys
print("Python version:", sys.version)

Python version: 3.10.14 (main, Mar 19 2024, 21:46:16) [Clang 15.0.0 (clang-1500.1.0.2.5)]


In [4]:
%pip install openai python-dotenv graphviz colorama

Note: you may need to restart the kernel to use updated packages.


In [5]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")

✅ All dependencies installed successfully!


In [6]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)

Updated sys.path: ['/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/glin/Documents/GitHub/M-LLAP/venv2/lib/python3.10/site-packages', '/Users/glin/Documents/GitHub/M-LLAP/design_patterns/src']


## The Agent Class

First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.

In [8]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


API Key set: True


In [9]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")

✅ Agent imported successfully!


Let's create some example agent, to see how it works.

In [14]:
# ---- Data Collection Agent ---- #
DC_Agent = Agent(
    name="Data Collection Agent",
    backstory="You collect 5 stocks of historical data OHLCV from S&P 500 Kaggle Dataset, and real-time stock data including external economic indicators.",
    task_description="Retrieve stock data from 2010-2023 and stream new data every 5 minutes.",
    task_expected_output="Formatted OHLCV data and external indicators, of 5 stocks."
)

In [ ]:
# ---- Feature Extraction Agent ---- #
FE_Agent = Agent(
    name="Feature Extraction Agent",
    backstory="You compute technical indicators from the collected 5 stock data.",
    task_description="Calculate MA, MACD, RSI, Bollinger Bands, and other technical indicators of 5 stocks.",
    task_expected_output="Feature matrix of OHLCV, external indicators, MA, MACD, RSI, Bollinger Bands, of 5 stocks."
)

In [ ]:
# ---- Model Training Agent ---- #
MT_Agent = Agent(
    name="Model Training Agent",
    backstory="You train stock prediction models using historical data (2010-2020) and validate with (2021-2023).",
    task_description="Train machine learning models (LSTM) and optimize hyperparameters.",
    task_expected_output="Prepare the input Feature matrix, with 3 financial math models and 3 machine learning models ready for inference, of 5 stocks."
)

In [ ]:
# ---- Prediction Generation Agent ---- #
PG_Agent = Agent(
    name="Prediction Generation Agent",
    backstory="You generate stock price predictions using the trained model.",
    task_description="Predict next-day and intra-day stock prices.",
    task_expected_output="Based on input and the 6 models, predict Stock price from 2024-1-1 to 2024-1-7, with confidence intervals, of 5 stocks."
)

In [ ]:
# ---- Integration Agent ---- #
IG_Agent = Agent(
    name="Integration Agent",
    backstory="You combine model predictions with external news sentiment and macroeconomic indicators.",
    task_description="Adjust predictions based on news impact and market conditions on year of 2023.",
    task_expected_output="Refined Stock price from 2024-1-1 to 2024-1-7, with adjusted confidence intervals, of 5 stocks. "
)

In [15]:
# ---- Alert Generation Agent ---- #
AG_Agent = Agent(
    name="Alert Generation Agent",
    backstory="You issue buy/sell/hold signals based on the model’s predictions and risk assessment.",
    task_description="Generate trading alerts and confidence scores.",
    task_expected_output="Based on the 6 models, provide actionable trade signals from 2024-1-1 to 2024-1-7, of 5 stocks."
)

You can also associate tools with the agent. Let's create a tool for writing some string into a CSV.

In [36]:
from src.tool_agent.tool import tool

In [37]:
@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")

In [16]:
agent_tool_example = Agent(
    name="Writer Agent",
    backstory="You are a language model specialised in writing text into .txt files",
    task_description="Write the string 'This is a Tool Agent' into './tool_agent_example.txt'",
    task_expected_output="A .txt file containing the given string",
    tools=write_str_to_txt,
)

In [17]:
agent_tool_example.run()


Thought: I need to write the string 'This is a Tool Agent' into a text file named 'tool_agent_example.txt'.

Using Tool: write_str_to_txt

Tool call dict: 
{'name': 'write_str_to_txt', 'arguments': {'string_data': 'This is a Tool Agent', 'txt_filename': 'tool_agent_example.txt'}, 'id': 0}
Data successfully written to tool_agent_example.txt

Tool result: 
None

Observations: {0: None}


"The string 'This is a Tool Agent' has been successfully written to 'tool_agent_example.txt'."

## Defining Agent Dependencies

Let's define two agents now.

We can define the agent dependencies using the `>>` operator.

In [19]:
# ---- Define Dependencies ---- #
DC_Agent >> FE_Agent >> MT_Agent  # Technical Analysis Flow
MT_Agent >> PG_Agent >> AG_Agent  # Model Chain
AG_Agent >> DC_Agent  # Feedback Loop (adaptive updates)

print("✅ Dependency Graph Established!")

✅ Dependency Graph Established!


In [21]:
print("Agent DC dependencies: ", DC_Agent.dependencies)
print("Agent DC dependents: ", DC_Agent.dependents)
print("Agent FE dependencies: ", FE_Agent.dependencies)
print("Agent FE dependents: ", FE_Agent.dependents)
print("Agent MT dependencies: ", MT_Agent.dependencies)
print("Agent MT dependents: ", MT_Agent.dependents)
print("Agent PG dependencies: ", PG_Agent.dependencies)
print("Agent PG dependents: ", PG_Agent.dependents)
print("Agent AG dependencies: ", AG_Agent.dependencies)
print("Agent AG dependents: ", AG_Agent.dependents)

Agent DC dependencies:  [Alert Generation Agent]
Agent DC dependents:  [Feature Extraction Agent]
Agent FE dependencies:  [Data Collection Agent]
Agent FE dependents:  [Model Training Agent]
Agent MT dependencies:  [Feature Extraction Agent]
Agent MT dependents:  [Prediction Generation Agent]
Agent PG dependencies:  [Model Training Agent]
Agent PG dependents:  [Alert Generation Agent]
Agent AG dependencies:  [Prediction Generation Agent]
Agent AG dependents:  [Data Collection Agent]


Now, if we run `DC_Agent`, the results will be added to `FE_Agent`'s context.

In [24]:
print(DC_Agent.run())

To complete the task of retrieving stock data from 2010-2023 and streaming new data every 5 minutes, I will provide a structured format for the OHLCV data along with external economic indicators. Here's how the output will be formatted:

1. **Historical Data (2010-2023):**
   - **Date:** YYYY-MM-DD
   - **Open:** Opening price of the stock on that day
   - **High:** Highest price of the stock on that day
   - **Low:** Lowest price of the stock on that day
   - **Close:** Closing price of the stock on that day
   - **Volume:** Number of shares traded on that day

2. **Real-Time Data (Streamed every 5 minutes):**
   - **Timestamp:** YYYY-MM-DD HH:MM:SS
   - **Open:** Opening price at the start of the 5-minute interval
   - **High:** Highest price during the 5-minute interval
   - **Low:** Lowest price during the 5-minute interval
   - **Close:** Closing price at the end of the 5-minute interval
   - **Volume:** Number of shares traded during the 5-minute interval

3. **External Economic 

In [25]:
print(FE_Agent.context)

Feature Extraction Agent received context: 
To accomplish the task of retrieving stock data from 2010-2023 and streaming new data every 5 minutes, I will outline a structured approach to provide formatted OHLCV data and external indicators.

1. **Historical Data Retrieval (2010-2023):**
   - Use a reliable financial data provider API (e.g., Alpha Vantage, Yahoo Finance, or Bloomberg) to fetch historical OHLCV data for the specified period.
   - Ensure the data includes Open, High, Low, Close prices, Volume, and any relevant external economic indicators such as interest rates, GDP growth rates, or unemployment rates.

2. **Data Formatting:**
   - Organize the historical data in a structured format, typically a CSV or a database table, with columns for Date, Open, High, Low, Close, Volume, and each external indicator.
   - Example format:
     ```
     Date, Open, High, Low, Close, Volume, Interest Rate, GDP Growth, Unemployment Rate
     2010-01-01, 100, 105, 95, 102, 1000000, 0.5, 2.5,

Now, if we run the second agent, it will use the context received from the previous agent to generate its output.

In [26]:
print(FE_Agent.run())

<Feature Matrix>
{
  "technical_indicators": [
    {
      "date": "YYYY-MM-DD",
      "moving_average": "value",
      "macd": "value",
      "rsi": "value",
      "bollinger_bands": {
        "upper_band": "value",
        "lower_band": "value"
      },
      "other_indicators": {
        "indicator_name_1": "value",
        "indicator_name_2": "value",
        ...
      }
    },
    ...
  ]
}
</Feature Matrix>

- **technical_indicators**: This section contains an array of objects, each representing a day's calculated technical indicators based on the OHLCV data and any external indicators provided.
- **moving_average**: The calculated moving average for the day.
- **macd**: The calculated Moving Average Convergence Divergence value for the day.
- **rsi**: The calculated Relative Strength Index value for the day.
- **bollinger_bands**: An object containing the upper and lower Bollinger Bands calculated for the day.
- **other_indicators**: A placeholder for any additional technical in

## The Crew

In [44]:
from src.multi_agent.crew import Crew

In [47]:
with Crew() as crew:
    # ---- Data Collection Agent ---- #
    DC_Agent = Agent(
        name="Data Collection Agent",
        backstory="You collect 5 stocks AAPL, MSFT, GOOGL, AMZN, TSLA, of historical data OHLCV from S&P 500 Kaggle Dataset at https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks, and real-time stock data including external economic indicators.",
        task_description="Retrieve stock data from 2023-10-01 to 2023-12-31.",
        task_expected_output="Formatted OHLCV data and external indicators, of 5 stocks. Do not omit, and provide full data. ",
    )

    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './writer1_agent.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )
    
    # ---- Feature Extraction Agent ---- #
    FE_Agent = Agent(
        name="Feature Extraction Agent",
        backstory="You compute technical indicators from the collected 5 stock data.",
        task_description="Calculate MA, MACD, RSI, Bollinger Bands, and other technical indicators of 5 stocks.",
        task_expected_output="Feature matrix of OHLCV, external indicators, MA, MACD, RSI, Bollinger Bands, of 5 stocks."
    )

    Writer2_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './writer2_agent.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )

    
    # ---- Model Training Agent ---- #
    MT_Agent = Agent(
        name="Model Training Agent",
        backstory="You train stock prediction models using historical data (from 2023-10-01 to 2023-11-30) and validate with (from 2023-12-1 to 2024-12-31).",
        task_description="use financial math and machine learning models and optimize hyperparameters.",
        task_expected_output="Prepare the input Feature matrix, with 3 financial math models and 3 machine learning models ready for inference, of 5 stocks."
    )
    
    # ---- Prediction Generation Agent ---- #
    PG_Agent = Agent(
        name="Prediction Generation Agent",
        backstory="You generate stock price predictions using the trained model.",
        task_description="Predict next-day and intra-day stock prices.",
        task_expected_output="Based on input and the 6 models, predict Stock price from 2024-1-1 to 2024-1-7, with confidence intervals, of 5 stocks for 6 models."
    )
    
    # Writer3_agent = Agent(
    #     name="Writer Agent",
    #     backstory="You are a language model specialised in writing text into .json files",
    #     task_description="Write the json response into './writer3_agent.json'",
    #     task_expected_output="A .json file containing the given string",
    #     tools=write_str_to_txt,
    # )
    
    # ---- Integration Agent ---- #
    IG_Agent = Agent(
        name="Integration Agent",
        backstory="You combine model predictions with external news sentiment and macroeconomic indicators.",
        task_description="Adjust predictions based on news impact and market conditions on year of 2023.",
        task_expected_output="Refined Stock price from 2024-1-1 to 2024-1-7, with adjusted confidence intervals, of 5 stocks. "
    )

    # Writer4_agent = Agent(
    #     name="Writer Agent",
    #     backstory="You are a language model specialised in writing text into .json files",
    #     task_description="Write the json response into './writer4_agent.json'",
    #     task_expected_output="A .json file containing the given string",
    #     tools=write_str_to_txt,
    # )

    # ---- Alert Generation Agent ---- #
    AG_Agent = Agent(
        name="Alert Generation Agent",
        backstory="You issue buy/sell/hold signals based on the model’s predictions and risk assessment.",
        task_description="Generate trading alerts and confidence scores.",
        task_expected_output="Based on the 6 models, provide actionable trade signals from 2024-1-1 to 2024-1-7, of 5 stocks."
    )

    # Writer5_agent = Agent(
    #     name="Writer Agent",
    #     backstory="You are a language model specialised in writing text into .json files",
    #     task_description="Write the json response into './writer5_agent.json'",
    #     task_expected_output="A .json file containing the given string",
    #     tools=write_str_to_txt,
    # )

    
    # ---- Define Dependencies ---- #
    DC_Agent >> Writer_agent >> FE_Agent >> Writer2_agent >> MT_Agent  # Technical Analysis Flow
    MT_Agent >> PG_Agent >> IG_Agent >> AG_Agent  # Model Chain
    #AG_Agent >> DC_Agent  # Feedback Loop (adaptive updates)



In [48]:
crew.run()


RUNNING AGENT: Data Collection Agent

<task_expected_output>
Unfortunately, I am unable to access real-time or future data beyond my training cut-off in October 2023. However, I can guide you on how to retrieve this data using available APIs or data providers. You can use financial data services like Alpha Vantage, Yahoo Finance, or Bloomberg to get the OHLCV data and external economic indicators for AAPL, MSFT, GOOGL, AMZN, and TSLA from 2023-10-01 to 2023-12-31. These platforms often provide comprehensive documentation on how to access and format the data you need.
</task_expected_output>

RUNNING AGENT: Writer Agent


Thought: I need to write the provided context string into a JSON file named './writer1_agent.json'.

Using Tool: write_str_to_txt

Tool call dict: 
{'name': 'write_str_to_txt', 'arguments': {'string_data': 'Unfortunately, I am unable to access real-time or future data beyond my training cut-off in October 2023. However, I can guide you on how to retrieve this data usi